In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import urljoin

def scrape_flags_from_table_layout():
    """
    【最終確定版】テーブルレイアウトに変更されたWikimedia Commonsのページ構造に対応したスクレイピングコード。
    """
    base_url = "https://commons.wikimedia.org/"
    list_page_url = urljoin(base_url, "wiki/Sovereign-state_flags")
    
    flag_data = []
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
    }

    try:
        # --- STEP 1: 一覧ページにアクセス ---
        print(f"一覧ページにアクセス中: {list_page_url}")
        response = requests.get(list_page_url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # ★★★★★ 新しいセレクタ ★★★★★
        # 各旗は 'display: inline-block' というスタイルが指定された <td> タグの中に格納されていることを特定。
        # このスタイルを持つ<td>を全て取得する。
        flag_containers = soup.select('td[style*="display: inline-block"]')
        
        if not flag_containers:
            print("エラー: 旗を格納しているコンテナが見つかりませんでした。")
            return None

        print(f"{len(flag_containers)} 件の旗が見つかりました。詳細ページを巡回します...")
        print("-" * 30)
        
        # --- STEP 2: 各コンテナを巡回して情報を抽出 ---
        for i, container in enumerate(flag_containers):
            # コンテナ内からファイルページへのリンク (class='mw-file-description') を探す
            file_link_tag = container.select_one('a.mw-file-description')
            
            # コンテナ内から国名が含まれるキャプション部分のリンクを探す
            # キャプション内の最後の<a>タグに国名が入っていることが多い
            name_link_tags = container.select('td > a')

            if not file_link_tag or not name_link_tags:
                continue
            
            # 国名とファイルページのURLを取得
            country_name = name_link_tags[-1].get_text(strip=True)
            file_page_url = urljoin(base_url, file_link_tag['href'])

            print(f"[{i+1}/{len(flag_containers)}] {country_name} を処理中...")

            try:
                time.sleep(0.5)
                
                file_page_response = requests.get(file_page_url, headers=headers)
                file_page_response.raise_for_status()
                file_soup = BeautifulSoup(file_page_response.content, 'html.parser')
                
                # オリジナルSVGファイルへのリンクを取得
                svg_link_tag = file_soup.select_one('div.fullMedia a')
                
                if svg_link_tag and svg_link_tag.has_attr('href'):
                    svg_url = svg_link_tag['href']
                    flag_data.append({
                        '国名': country_name,
                        'SVG画像URL': svg_url
                    })
                    print(f"  └── SUCCESS: SVG画像のURLを取得しました。")
                else:
                    print(f"  └── WARNING: SVG画像へのリンクが見つかりませんでした。")

            except Exception as e:
                print(f"  └── ERROR: 詳細ページの処理中にエラーが発生しました: {e}")

        df = pd.DataFrame(flag_data)
        return df

    except Exception as e:
        print(f"致命的なエラーが発生しました: {e}")
        return None

# --- メインの実行部分 ---
if __name__ == "__main__":
    flags_df = scrape_flags_from_table_layout()
    
    if flags_df is not None and not flags_df.empty:
        print("\n" + "="*30)
        print("スクレイピングが正常に完了しました。")
        print("--- 取得データサンプル ---")
        print(flags_df.head())
        
        print(f"\n合計 {len(flags_df)} 件のデータを取得しました。")
        
        try:
            flags_df.to_csv('flags_database_success.csv', index=False, encoding='utf-8-sig')
            print("\nデータを 'flags_database_success.csv' として保存しました。")
        except Exception as e:
            print(f"ファイルへの保存中にエラーが発生しました: {e}")
    else:
        print("\n" + "="*30)
        print("データを取得できませんでした。上記のログを確認してください。")